## Advanced Box Score Calculations
Author: Akshay Easwaran <akeaswaran@me.com>

---
Based on work by ESPN's Bill Connelly, Football Outsiders, and Football Study Hall

Data from CollegeFootballData.com

Useful Resources for Stat Definitions:
- Football Study Hall website: https://www.footballstudyhall.com/
- Football Study Hall book: https://www.amazon.com/Study-Hall-College-Football-Stories/dp/1484989961
- Football Outsiders: https://www.footballoutsiders.com/info/glossary
- Example Advanced Box Score: https://twitter.com/ESPN_BillC/status/1176572006969597954/photo/1

**How to use this Notebook**

1. Dump a game's play by play data and drive data from CollegeFootballData.com.
2. Change the file names in the next cell to match where your data files are located.
3. Change the away and home team names appropriately.
4. Hit Cell > Run All Cells.

In [86]:
import pandas as pd

pbp_data = pd.read_json('data/pbp/pbp-data-unc-gatech.json')
# pbp_data.info()

drives = pd.read_json('data/drives/drive-data-unc-gatech.json')
drives.drop(['offense_conference','start_time','end_time','defense_conference','elapsed','game_id'], axis = 1, inplace=True) 
drives = drives[
    ~drives.drive_result.isin(['END OF HALF','END OF GAME'])
]

away_team = 'North Carolina'
home_team = 'Georgia Tech'

In [87]:
# Data Cleaning
# Fix the bad yard line markers for away teams
drives.loc[
    drives.offense == away_team, ['start_yardline']
] = 100 - drives.start_yardline
drives.loc[
    drives.offense == away_team, ['end_yardline']
] = 100 - drives.end_yardline

print("Total Drives:", len(drives))

# Eliminate garbage time plays
garbage_time_data = pbp_data[
    (pbp_data.down == 0)
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 38) & (pbp_data.period == 2))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 28) & (pbp_data.period == 3))
    | ((abs(pbp_data.offense_score - pbp_data.defense_score) >= 22) & (pbp_data.period == 4))
]

print("Total Plays:", len(pbp_data))
print("Garbage Time Plays:", len(garbage_time_data))
pbp_data.drop(garbage_time_data.index, inplace=True)
print("Non-Garbage Time Plays:", len(pbp_data))

# Drop some unnecessary columns
pbp_data.drop(['offense_conference','defense_conference','clock'], axis = 1, inplace=True) 
# Ignore some types of plays cause they're special teams and weird
ignore_types = ["Defensive 2pt Conversion","Blocked Field Goal","Blocked Punt","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]
pbp_data = pbp_data[~(pbp_data.play_type.isin(ignore_types))]
print("Total clean plays:", len(pbp_data))

Total Drives: 24
Total Plays: 191
Garbage Time Plays: 10
Non-Garbage Time Plays: 181
Total clean plays: 145


In [88]:
import numpy as np

pbp_data.distance = pbp_data.distance.astype(float)

bad_types = ["Interception","Pass Interception Return","Interception Return Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

def is_successful(down, distance, yards_gained, play_type):
    if (play_type in bad_types):
        return False 
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp_data['play_successful'] = np.vectorize(is_successful)(pbp_data.down, pbp_data.distance, pbp_data.yards_gained, pbp_data.play_type)
pbp_data.play_successful.value_counts()

def is_explosive(yards_gained):
    if (yards_gained >= 15):
        return True
    else:
        return False
    
pbp_data['play_explosive'] = np.vectorize(is_explosive)(pbp_data.yards_gained)

In [89]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown","Interception","Pass Interception Return","Interception Return Touchdown"]
rush_types = ["Rush","Rushing Touchdown",'Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']

print("Success Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team)]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_successful == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team)]))

Success Rates
North Carolina
Overall: 0.4479166666666667
Passing: 0.2604166666666667
Rushing: 0.1875
Georgia Tech
Overall: 0.3673469387755102
Passing: 0.1836734693877551
Rushing: 0.1836734693877551


In [90]:
# Explosiveness rates
# Defined as rate of 15+ yards gains

print("Explosiveness Rates")
print(away_team)
print("Overall:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == away_team)]))
print("Rushing:",len(pbp_data[(pbp_data.offense == away_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == away_team)]))
print(home_team)
print("Overall:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True)]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Passing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(pass_types))]) / len(pbp_data[(pbp_data.offense == home_team)]))
print("Rushing:",len(pbp_data[(pbp_data.offense == home_team) & (pbp_data.play_explosive == True) & (pbp_data.play_type.isin(rush_types))]) / len(pbp_data[(pbp_data.offense == home_team)]))

Explosiveness Rates
North Carolina
Overall: 0.125
Passing: 0.09375
Rushing: 0.03125
Georgia Tech
Overall: 0.14285714285714285
Passing: 0.08163265306122448
Rushing: 0.061224489795918366


In [91]:
# Havoc plays
havoc_plays = pbp_data[
    (((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type == 'Fumble Recovery (Opponent)')
    | (pbp_data.play_type == 'Sack')
    | (pbp_data.play_type.str.contains('Interception', regex=False))
    | (pbp_data.yards_gained < 0))
    & (pbp_data.play_type != 'Penalty')
]

print(away_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == away_team
]), "/", len(pbp_data[
    pbp_data.defense == away_team
]), "(",len(havoc_plays[
    havoc_plays.defense == away_team
]) / len(pbp_data[
    pbp_data.defense == away_team
]),")")

print(home_team,"Havoc Rate: ", len(havoc_plays[
    havoc_plays.defense == home_team
]), "/", len(pbp_data[
    pbp_data.defense == home_team
]), "(",len(havoc_plays[
    havoc_plays.defense == home_team
]) / len(pbp_data[
    pbp_data.defense == home_team
]),")")

North Carolina Havoc Rate:  10 / 49 ( 0.20408163265306123 )
Georgia Tech Havoc Rate:  13 / 96 ( 0.13541666666666666 )


In [92]:
# Calculating converted 3rd and 8+s
ignore_turnovers = ['Fumble Recovery (Opponent)','Fumble Return Touchdown',"Interception","Pass Interception Return","Interception Return Touchdown"]

third_longs = pbp_data[
    (pbp_data.down == 3)
    & (pbp_data.distance >= 8)
    & (~pbp_data.play_type.isin(ignore_turnovers))
]
# converted_3rd_longs

away_team_thirds = third_longs[
    (third_longs.offense == away_team)
]

away_team_thirds_converted = away_team_thirds[
    (away_team_thirds.yards_gained >= 8)
]
print("Converted 3rd and 8+ for", away_team,":",len(away_team_thirds_converted),"of",len(away_team_thirds))

home_team_thirds = third_longs[
    (third_longs.offense == home_team)
]

home_team_thirds_converted = home_team_thirds[
    (home_team_thirds.yards_gained >= 8)
]

print("Converted 3rd and 8+ for", home_team,":",len(home_team_thirds_converted),"of",len(home_team_thirds))

Converted 3rd and 8+ for North Carolina : 5 of 10
Converted 3rd and 8+ for Georgia Tech : 1 of 6


In [93]:
away_team_drives = drives[
    drives.offense == away_team
]

print(away_team, "Drives:", len(away_team_drives))
print(away_team,"Yards:",sum(away_team_drives.yards))
print(away_team,"Plays:",sum(away_team_drives.plays))
print(away_team, "Avg Starting Field Position:", sum(away_team_drives.start_yardline) / len(away_team_drives))
print(away_team, "Yards per Play:", sum(away_team_drives.yards) / sum(away_team_drives.plays))
print(away_team, "Plays per Drive:", sum(away_team_drives.plays) / len(away_team_drives))
print(away_team, "Yards per Drive:", sum(away_team_drives.yards) / len(away_team_drives))

North Carolina Drives: 12
North Carolina Yards: 548
North Carolina Plays: 98
North Carolina Avg Starting Field Position: 28.416666666666668
North Carolina Yards per Play: 5.591836734693878
North Carolina Plays per Drive: 8.166666666666666
North Carolina Yards per Drive: 45.666666666666664


In [114]:
home_team_drives = drives[
    drives.offense == home_team
]

print(home_team, "Drives:", len(home_team_drives))
print(home_team,"Yards:",sum(home_team_drives.yards))
print(home_team,"Plays:",sum(home_team_drives.plays))
print(home_team, "Avg Starting Field Position:", sum(home_team_drives.start_yardline) / len(home_team_drives))
print(home_team, "Yards per Play:", sum(home_team_drives.yards) / sum(home_team_drives.plays))
print(home_team, "Plays per Drive:", sum(home_team_drives.plays) / len(home_team_drives))
print(home_team, "Yards per Drive:", sum(home_team_drives.yards) / len(home_team_drives))

Georgia Tech Drives: 12
Georgia Tech Yards: 317
Georgia Tech Plays: 52
Georgia Tech Avg Starting Field Position: 27.083333333333332
Georgia Tech Yards per Play: 6.096153846153846
Georgia Tech Plays per Drive: 4.333333333333333
Georgia Tech Yards per Drive: 26.416666666666668


In [95]:
# Measuring success rate for a single player
# pbp_data[
#     (pbp_data.play_text.str.contains("Quentin Harris"))
#     & (pbp_data.play_type.isin(pass_types))
#     & (~pbp_data.play_type.str.contains("Sack"))
# ].play_successful.value_counts(normalize=True)

In [96]:
# Standard vs Passing Downs success rates
# Success rate on standard downs == leverage rate

standard_downs = pbp_data[
    (pbp_data.down == 1)
    | ((pbp_data.down == 2) & (pbp_data.distance <= 7))
    | ((pbp_data.down == 3) & (pbp_data.distance <= 4))
    | ((pbp_data.down == 4) & (pbp_data.distance <= 4)) 
]

passing_downs = pbp_data[
    ((pbp_data.down == 2) & (pbp_data.distance >= 8))
    | ((pbp_data.down == 3) & (pbp_data.distance >= 5))
    | ((pbp_data.down == 4) & (pbp_data.distance >= 5)) 
]

pass_plays = pbp_data[
    pbp_data.play_type.isin(pass_types)
]
rush_plays = pbp_data[
    pbp_data.play_type.isin(rush_types)
]

In [97]:
print(away_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == away_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == away_team)
]))

North Carolina Success Rate on Std Downs: 0.4482758620689655


In [98]:
print(away_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == away_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == away_team)
]))

North Carolina Success Rate on Passing Downs: 0.4473684210526316


In [99]:
print(home_team,"Success Rate on Std Downs:",len(standard_downs[
    (standard_downs.offense == home_team)
    & (standard_downs.play_successful == True)
]) / len(standard_downs[
    (standard_downs.offense == home_team)
]))

Georgia Tech Success Rate on Std Downs: 0.40625


In [100]:
print(home_team,"Success Rate on Passing Downs:",len(passing_downs[
    (passing_downs.offense == home_team)
    & (passing_downs.play_successful == True)
]) / len(passing_downs[
    (passing_downs.offense == home_team)
]))

Georgia Tech Success Rate on Passing Downs: 0.29411764705882354


In [101]:
# Stuff Rate
stuffed_plays = rush_plays[
    (rush_plays.yards_gained <= 0)
    & (rush_plays.play_type != 'Sack')
]

In [102]:
away_team_stuffs = stuffed_plays[
    stuffed_plays.defense == away_team
]
print(away_team,"Defensive Stuff Rate: ",len(away_team_stuffs),"/",len(rush_plays[rush_plays.defense == away_team]),"(",len(away_team_stuffs)/len(rush_plays[rush_plays.defense == away_team]),")")

North Carolina Defensive Stuff Rate:  8 / 26 ( 0.3076923076923077 )


In [103]:
home_team_stuffs = stuffed_plays[
    stuffed_plays.defense == home_team
]
print(home_team,"Defensive Stuff Rate: ",len(home_team_stuffs),"/",len(rush_plays[rush_plays.defense == home_team]),"(",len(home_team_stuffs)/len(rush_plays[rush_plays.defense == home_team]),")")

Georgia Tech Defensive Stuff Rate:  12 / 44 ( 0.2727272727272727 )


In [104]:
# Opportunity Rate
rush_opps = rush_plays[
    (rush_plays.yards_gained >= 5)
]
print(away_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == away_team]),"/",len(rush_plays[rush_plays.offense == away_team]),"(",len(rush_opps[rush_opps.offense == away_team])/len(rush_plays[rush_plays.offense == away_team]),")")

North Carolina Rush Opp Rate:  18 / 44 ( 0.4090909090909091 )


In [105]:
print(home_team,"Rush Opp Rate: ",len(rush_opps[rush_opps.offense == home_team]),"/",len(rush_plays[rush_plays.offense == home_team]),"(",len(rush_opps[rush_opps.offense == home_team])/len(rush_plays[rush_plays.offense == home_team]),")")

Georgia Tech Rush Opp Rate:  8 / 26 ( 0.3076923076923077 )


In [106]:
# Line Yards -- not adjusted for down/distance/opponent/shotgun
def adjust_strength_for_ol(yards_gained):
    if (yards_gained < 0):
        return yards_gained * 1.25
    elif (yards_gained >= 0 & yards_gained <= 3):
        return yards_gained * 1.0
    elif (yards_gained >= 4 & yards_gained <= 6):
        return yards_gained * 0.5
    else:
        return 0

rush_plays['line_yards'] = rush_plays.apply(lambda x: adjust_strength_for_ol(x.yards_gained), axis=1)
rush_plays['highlight_yards'] = rush_plays.apply(lambda x: x.yards_gained - adjust_strength_for_ol(x.yards_gained), axis=1)
print(away_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].line_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].highlight_yards) / len(rush_plays[rush_plays.offense == away_team]))
print(away_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == away_team].yards_gained) / len(rush_plays[rush_plays.offense == away_team]))
print(home_team,"Line Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].line_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Highlight Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].highlight_yards) / len(rush_plays[rush_plays.offense == home_team]))
print(home_team,"Yards per Carry:",sum(rush_plays[rush_plays.offense == home_team].yards_gained) / len(rush_plays[rush_plays.offense == home_team]))

North Carolina Line Yards per Carry: 5.119318181818182
North Carolina Highlight Yards per Carry: 0.10795454545454546
North Carolina Yards per Carry: 5.2272727272727275
Georgia Tech Line Yards per Carry: 4.451923076923077
Georgia Tech Highlight Yards per Carry: 0.20192307692307693
Georgia Tech Yards per Carry: 4.653846153846154


/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/akeaswaran/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [107]:
# Expected and Adjusted Turnover Margins

adj_turnover_plays = pbp_data[
    (pbp_data.play_type.str.contains('Interception', regex=False))
    | ((pbp_data.play_type == 'Pass Incompletion')
    & (pbp_data.play_text.str.contains('broken up', regex=False)))
    | (pbp_data.play_type.str.contains('Fumble', regex=False))
]

adj_fum = 0.5 * len(adj_turnover_plays[
    (adj_turnover_plays.play_type.str.contains('Fumble', regex=False))
])

# away_team Adj Turnovers
away_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == away_team)
    | (adj_turnover_plays.defense == away_team)
]
print(away_team)
away_team_ints_def = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_ints_off = len(away_team_tos[
   (away_team_tos.play_type.str.contains('Interception', regex=False))
    & (away_team_tos.offense == away_team)
])

away_team_pds = len(away_team_tos[
   (away_team_tos.play_type == 'Pass Incompletion')
    & (away_team_tos.play_text.str.contains('broken up', regex=False))
    & (away_team_tos.defense == away_team)
])

away_team_fum_rec = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Own)') & (away_team_tos.offense == away_team)]
away_team_fum_lost = away_team_tos[(away_team_tos.play_type == 'Fumble Recovery (Opponent)') & (away_team_tos.offense == away_team)]

print("Def INTs:",away_team_ints_def)
print("Off INTs:",away_team_ints_off)
print("PDs:",away_team_pds)
print("Exp INTs:",0.21 * away_team_pds)
print("Fum Recovered:",len(away_team_fum_rec))
print("Fum Lost:",len(away_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Actual TO:",away_team_ints_off + len(away_team_fum_lost))
print("Actual TO Margin:",(away_team_ints_def - away_team_ints_off) - len(away_team_fum_lost))
away_team_exp_to = 0.21 * away_team_pds + adj_fum
print("Exp TO:",away_team_exp_to)

North Carolina
Def INTs: 2
Off INTs: 1
PDs: 1
Exp INTs: 0.21
Fum Recovered: 1
Fum Lost: 0
Exp Fum: 1.0
Actual TO: 1
Actual TO Margin: 1
Exp TO: 1.21


In [108]:
# home_team Adj Turnovers
home_team_tos = adj_turnover_plays[
    (adj_turnover_plays.offense == home_team)
    | (adj_turnover_plays.defense == home_team)
]
print(home_team)
home_team_ints_def = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_ints_off = len(home_team_tos[
   (home_team_tos.play_type.str.contains('Interception', regex=False))
    & (home_team_tos.offense == home_team)
])

home_team_pds = len(home_team_tos[
   (home_team_tos.play_type == 'Pass Incompletion')
    & (home_team_tos.play_text.str.contains('broken up', regex=False))
    & (home_team_tos.defense == home_team)
])

home_team_fum_rec = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Own)') & (home_team_tos.offense == home_team)]
home_team_fum_lost = home_team_tos[(home_team_tos.play_type == 'Fumble Recovery (Opponent)') & (home_team_tos.offense == home_team)]

print("Def INTs:",home_team_ints_def)
print("Off INTs:",home_team_ints_off)
print("PDs:",home_team_pds)
print("Exp INTs:",0.21 * home_team_pds)
print("Fum Recovered:",len(home_team_fum_rec))
print("Fum Lost:",len(home_team_fum_lost))
print("Exp Fum:",adj_fum)
print("Exp TO:",0.21 * home_team_pds + adj_fum)
print("Actual TO:",home_team_ints_off + len(home_team_fum_lost))
print("Actual TO Margin:",(home_team_ints_def - home_team_ints_off) - len(home_team_fum_lost))
home_team_exp_to = 0.21 * home_team_pds + adj_fum
print("Exp TO:",home_team_exp_to)

Georgia Tech
Def INTs: 1
Off INTs: 2
PDs: 3
Exp INTs: 0.63
Fum Recovered: 1
Fum Lost: 0
Exp Fum: 1.0
Exp TO: 1.63
Actual TO: 2
Actual TO Margin: -1
Exp TO: 1.63


In [109]:
away_team_exp_to_margin = (away_team_exp_to - home_team_exp_to)
print("Exp TO Margin for",away_team,":", away_team_exp_to_margin)
home_team_exp_to_margin = (home_team_exp_to - away_team_exp_to)
print("Exp TO Margin for",home_team,":", home_team_exp_to_margin)

Exp TO Margin for North Carolina : -0.41999999999999993
Exp TO Margin for Georgia Tech : 0.41999999999999993


In [110]:
print("TO Luck (pts) for",away_team,":", (((away_team_ints_def - away_team_ints_off) - len(away_team_fum_lost)) - away_team_exp_to_margin) * 5.0)
print("TO Luck (pts) for",home_team,":", (((home_team_ints_def - home_team_ints_off) - len(home_team_fum_lost)) - home_team_exp_to_margin) * 5.0)

TO Luck (pts) for North Carolina : 7.1
TO Luck (pts) for Georgia Tech : -7.1


In [111]:
# Scoring Opportunities
# Definition: roughly, any time you get inside the opponent's 35, you should probably score

scoring_opps = drives[
    ((drives.start_yardline + drives.yards) >= 65)
]

# away team's scoring opps
print("Scoring Opportunities (IE: Drives inside Opponent's 35)")
print(away_team)
away_team_scoring_opps = scoring_opps[
    scoring_opps.offense == away_team
]
print("Total:",len(away_team_scoring_opps))
print("Scored:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(away_team_scoring_opps[away_team_scoring_opps.scoring == True]) / len(away_team_scoring_opps))
print("Opps/Drive:",len(away_team_scoring_opps) / len(drives[drives.offense == away_team]))
print("Points/Opp:",(len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'TD']) * 7 + len(away_team_scoring_opps[away_team_scoring_opps.drive_result == 'FG'] * 3)) / len(away_team_scoring_opps))

print("")
# home team's scoring opps
print(home_team)
home_team_scoring_opps = scoring_opps[
    scoring_opps.offense == home_team
]
print("Total:",len(home_team_scoring_opps))
print("Scored:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]))
print("Opp Efficiency:",len(home_team_scoring_opps[home_team_scoring_opps.scoring == True]) / len(home_team_scoring_opps))
print("Opps/Drive:",len(home_team_scoring_opps) / len(drives[drives.offense == home_team]))
print("Points/Opp:",(len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'TD']) * 7 + len(home_team_scoring_opps[home_team_scoring_opps.drive_result == 'FG'] * 3)) / len(home_team_scoring_opps))

Scoring Opportunities (IE: Drives inside Opponent's 35)
North Carolina
Total: 7
Scored: 6
Opp Efficiency: 0.8571428571428571
Opps/Drive: 0.5833333333333334
Points/Opp: 5.142857142857143

Georgia Tech
Total: 3
Scored: 3
Opp Efficiency: 1.0
Opps/Drive: 0.25
Points/Opp: 7.0


In [112]:
# Rushing Stats
rush_carries = rush_plays[
    (~rush_plays.play_type.isin(['Fumble Recovery (Opponent)','Sack','Fumble Return Touchdown']))
]

# look these up manually
away_team_rushers = ["Kenny Pickett","Todd Sibley Jr.","V'Lique Carter","Shocky Jacques-Louis","Maurice Ffrench"]
home_team_rushers = ["Quentin Harris","Deon Jackson","Mataeo Durant","Jordan Waters"]

def verify_division(num1, num2):
    if num2 == 0:
        return 0
    else:
        return num1 / num2

print(away_team,"Rushing Stats")
for qb in away_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True)]), len(pbp_data[(pbp_data.play_text.str.contains(qb))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")

print("")

print(home_team,"Rushing Stats")
for qb in home_team_rushers:
    print(qb + ":", 
          len(rush_carries[rush_carries.play_text.str.contains(qb)]),"car",
          sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained),"yards",
          len(rush_carries[(rush_carries.play_text.str.contains(qb)) & (rush_carries.play_type == "Rushing Touchdown")]),"TDs",
          len(rush_plays[(rush_plays.play_text.str.contains(qb)) & (rush_plays.play_type.str.contains("Fumble"))]),"Fum",
          "(",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"YPC",",",
          verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].line_yards), len(rush_carries[rush_carries.play_text.str.contains(qb)])),"line YPC",",",
          "SR%:",verify_division(len(pbp_data[(pbp_data.play_text.str.contains(qb)) & (pbp_data.play_successful == True)]), len(pbp_data[(pbp_data.play_text.str.contains(qb))])),",",
          "Opp Rate:",verify_division(len(rush_opps[rush_opps.play_text.str.contains(qb)]), len(rush_carries[rush_carries.play_text.str.contains(qb)])),",",
          "Hlt/Opp:",verify_division(sum(rush_carries[rush_carries.play_text.str.contains(qb)].yards_gained), len(rush_opps[rush_opps.play_text.str.contains(qb)])),
          ")")


North Carolina Rushing Stats
Kenny Pickett: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Todd Sibley Jr.: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
V'Lique Carter: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Shocky Jacques-Louis: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Maurice Ffrench: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )

Georgia Tech Rushing Stats
Quentin Harris: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Deon Jackson: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Mataeo Durant: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )
Jordan Waters: 0 car 0 yards 0 TDs 0 Fum ( 0 YPC , 0 line YPC , SR%: 0 , Opp Rate: 0 , Hlt/Opp: 0 )


In [113]:
# Passing Stats
pass_attempts = pass_plays[
    (pass_plays.play_type == 'Pass Reception')
    | (pass_plays.play_type == 'Pass Incompletion')
    | (pass_plays.play_type.str.contains('Interception'))
]

# look these up manually
away_team_qbs = ["Kenny Pickett"]
home_team_qbs = ["Quentin Harris"]

print(away_team,"Passing Stats")
for qb in away_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")
print("")
print(home_team,"Passing Stats")    
for qb in home_team_qbs:
    print(qb + ":", 
          len(pass_completions[pass_completions.play_text.str.contains(qb)]),
          "/",len(pass_attempts[pass_attempts.play_text.str.contains(qb)]),
          sum(pass_completions[pass_completions.play_text.str.contains(qb)].yards_gained),"yards",
          len(pass_completions[(pass_completions.play_text.str.contains(qb)) & (pass_completions.play_type == "Passing Touchdown")]),"TDs",
          len(pass_attempts[(pass_attempts.play_text.str.contains(qb)) & (pass_attempts.play_type.str.contains("Interception"))]),"INTs")


North Carolina Passing Stats
Kenny Pickett: 28 / 0 225 yards 3 TDs 0 INTs

Georgia Tech Passing Stats
Quentin Harris: 18 / 0 180 yards 1 TDs 0 INTs
